# Corridor Comparison: I-680

In [1]:
import pandas as pd
from siuba import *
from calitp_data_analysis.sql import to_snakecase

### Importing the data
For this analysis, we need to read in data from:
* Caltrans Traffic Census Program: Using 2022 AADT Data
* Streetlight: Using Segment Analysis, Network Performance, and AADT for 2022
* Replica: Using AADT Count Application

In [20]:
ct_aadt = to_snakecase(pd.read_excel(
    "https://dot.ca.gov/-/media/dot-media/programs/traffic-operations/documents/census/2022/2022-traffic-volumes-ca.xlsx",
    sheet_name = "2022 AADT DATA"
    ))

In [21]:
ct_aadt.sample()

,district,route,route_sfx,county,pm_pfx,pm,pm_sfx,location_description,back_peak_hour,back_peak_madt,back_aadt,ahead_peak_hour,ahead_peak_madt,ahead_aadt
3190,4,80,NaN,CC,NaN,0.0,NaN,ALAMEDA/CONTRA COSTA COUNTY LINE,NaN,NaN,NaN,12100.0,186000.0,181000.0


In [38]:
## Streetlight AADT
sl_aadt = to_snakecase(pd.read_csv("gs://calitp-analytics-data/data-analyses/big_data/i680/1751074_I680_AADT_Test_estimated_aadt.csv"))

In [23]:
## Streetlight Network Performance Test
sl_npt = to_snakecase(pd.read_csv("gs://calitp-analytics-data/data-analyses/big_data/i680/1751071_I680_network_performance_test_network_performance.csv"))

In [24]:
## Streetlight Segment Analysis
sl_sa = to_snakecase(pd.read_csv("gs://calitp-analytics-data/data-analyses/big_data/i680/1751075_i680_Segment_Analysis_test_sa_all.csv"))

In [26]:
## Joining the streetlight segments so that all STreetlight Analyses are together

In [39]:
sl_aadt = (sl_aadt>>select(_.data_periods, _.zone_id, _.zone_name,
                           _.estimated_2022_aadt, _.lower_95_percent_prediction_interval,
                           _.upper_95_percent_prediction_interval))

In [40]:
sl_aadt.columns = ['aadt_' + str(col) for col in sl_aadt.columns]


In [30]:
sl_npt = (sl_npt>>filter(_.day_part=="0: All Day (12am-12am)")>>select(_.data_periods, _.zone_name, _.zone_id,
                                            _.average_daily_segment_traffic__stl_volume_))

In [42]:
sl_npt.columns = ['npt_' + str(col) for col in sl_npt.columns]

In [ ]:
sl_sa = ((sl_sa>>filter(_.day_part=="0: All Day (12am-12am)")>>select(_.data_periods, _.zone_id, 
                                                                    _.zone_name,
                                                                     _.average_daily_segment_traffic__stl_volume_)))

In [55]:
sl_sa.columns = ['sa' + str(col) for col in sl_sa.columns]

In [58]:
pd.merge(sl_aadt, sl_npt, left_on="aadt_zone_name", right_on="npt_zone_name")


,aadt_data_periods,aadt_zone_id,aadt_zone_name,aadt_estimated_2022_aadt,aadt_lower_95_percent_prediction_interval,aadt_upper_95_percent_prediction_interval,npt_data_periods,npt_zone_name,npt_zone_id,npt_average_daily_segment_traffic__stl_volume_
0,"Jan 01, 2022 - Dec 31, 2022",NaN,Benicia-Martinez Bridge / 123867334 / 1,39493,31800,48800,"Oct 01, 2021 - Sep 30, 2022",Benicia-Martinez Bridge / 123867334 / 1,NaN,35593
1,"Jan 01, 2022 - Dec 31, 2022",NaN,Benicia-Martinez Bridge / 439915393 / 1,38653,31100,47800,"Oct 01, 2021 - Sep 30, 2022",Benicia-Martinez Bridge / 439915393 / 1,NaN,35465
2,"Jan 01, 2022 - Dec 31, 2022",NaN,Benicia-Martinez Bridge / 4968908 / 1,57224,46500,69000,"Oct 01, 2021 - Sep 30, 2022",Benicia-Martinez Bridge / 4968908 / 1,NaN,57498
3,"Jan 01, 2022 - Dec 31, 2022",NaN,Benicia-Martinez Bridge / 52000448 / 1,60529,49200,73000,"Oct 01, 2021 - Sep 30, 2022",Benicia-Martinez Bridge / 52000448 / 1,NaN,61440
4,"Jan 01, 2022 - Dec 31, 2022",NaN,Benicia-Martinez Bridge / 8703049 / 1,38663,31100,47800,"Oct 01, 2021 - Sep 30, 2022",Benicia-Martinez Bridge / 8703049 / 1,NaN,35173
...,...,...,...,...,...,...,...,...,...,...
556,"Jan 01, 2022 - Dec 31, 2022",NaN,Sinclair Freeway / 961140461 / 1,84762,70000,99800,"Oct 01, 2021 - Sep 30, 2022",Sinclair Freeway / 961140461 / 1,NaN,88248
557,"Jan 01, 2022 - Dec 31, 2022",NaN,Sinclair Freeway / 961140462 / 1,88021,72700,103600,"Oct 01, 2021 - Sep 30, 2022",Sinclair Freeway / 961140462 / 1,NaN,95175
558,"Jan 01, 2022 - Dec 31, 2022",NaN,Sinclair Freeway / 961140463 / 1,87816,72500,103400,"Oct 01, 2021 - Sep 30, 2022",Sinclair Freeway / 961140463 / 1,NaN,80341
559,"Jan 01, 2022 - Dec 31, 2022",NaN,Sinclair Freeway / 986291461 / 1,57891,47000,69800,"Oct 01, 2021 - Sep 30, 2022",Sinclair Freeway / 986291461 / 1,NaN,48800
